In [14]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import glob
import pandas as pd

from astropy.io import fits

matplotlib.rc('xtick', labelsize=15) 
matplotlib.rc('ytick', labelsize=15) 

In [27]:
# Import the comparison data (work on this later...)
# hdus = fits.open('/Users/xwou/Documents/Research/utilities/smhr-rpa/smh/data/spectra/cd-38_245.fits')
# head = hdus[0].header
# data = hdus[0].data
# hdus.close()
# Fits file only included flux data; wavelength is stored as a starting point plus delta wl along the axis

In [5]:
# Import the data
name_spec = ['smss1_09','smss1_12','smss1_18','smss1_29','smss1_33',
             'smss338','smss343','smss697','smss709','smss710','smss712']

old_spec, new_spec = [], []
for i in range(len(name_spec)):
    old_spec.append(pd.read_csv('../data/norm_spec/'+name_spec[i]+'_old',delim_whitespace=True))
    try:
        new_spec.append(pd.read_csv('../data/norm_spec/'+name_spec[i]+'_rered',delim_whitespace=True))
    except:
        tmp_spec = pd.read_csv('../data/norm_spec/'+name_spec[i]+'_old',delim_whitespace=True)
        tmp_spec.loc[:,'flux'] = np.zeros(len(tmp_spec))
        new_spec.append(tmp_spec.copy())

In [12]:
# Define the plotting function
# A4 canvas
fig_width_cm = 21                                # A4 page
fig_height_cm = 29.7
inches_per_cm = 1 / 2.54                         # Convert cm to inches
fig_width = fig_width_cm * inches_per_cm         # width in inches
fig_height = fig_height_cm * inches_per_cm       # height in inches
fig_size = [fig_width, fig_height]

# Wavelength ranges for the summary plot
start_wls = [5163]
end_wls = [5187]
elem_names = ['Mg']

def plot_mg_stack(old_data,new_data,filename,start_wls=start_wls,end_wls=end_wls,elem_names=elem_names):
    # Determine how many rows and subplots in total needed
    assert (len(old_data) == len(new_data)) & (len(start_wls) == len(end_wls))
    N_page = len(start_wls)
    N_stack = len(old_data)
    
    # Loop through pages with each pages with a given amount of rows
    for i in range(N_page):
        f = plt.figure(figsize=fig_size)
        # For each page, plot rows and save it to a separate pdf file

        # Start plotting
        for j in range(N_stack):
            # Determine the starting index and ending index for a given stack
            # Could just plot the whole thing all the time, but we want to save time
            start_wl, end_wl = start_wls[i], end_wls[i]

            # Plot the old spectrum
            if j in range(0,5):
                try:
                    start_ind = np.where(abs(old_data[j]['wavelength']-start_wl) <= 0.1)[0][0]
                    end_ind = np.where(abs(old_data[j]['wavelength']-end_wl) <= 0.1)[0][-1]
                    plt.plot(old_data[j]['wavelength'][start_ind:end_ind],old_data[j]['flux'][start_ind:end_ind]+j,
                             ls='-',c='k',lw=0.5,label=name_spec[j])
                except IndexError:
                    print('Star',name_spec[i],'has no old spectrum at wavelength range:',str(start_wl),'to',str(end_wl))

            # Plot the new re-reduced spectrum
            else: 
                try:
                    start_ind = np.where(abs(new_data[j]['wavelength']-start_wl) <= 0.1)[0][0]
                    end_ind = np.where(abs(new_data[j]['wavelength']-end_wl) <= 0.1)[0][-1]
                    plt.plot(new_data[j]['wavelength'][start_ind:end_ind],new_data[j]['flux'][start_ind:end_ind]+j,
                             ls='--',c='r',lw=0.5,label=name_spec[j])
                except IndexError:
                    print('Star',name_spec[i],'has no rereduced spectrum at wavelength range:',str(start_wl),'to',str(end_wl))
                

        plt.xlim([start_wl,end_wl])
        plt.ylim([0,j+1.2])
        plt.xlabel(r'Wavelength [$\mathring{A}$]',fontsize=20)
        plt.ylabel('Flux',fontsize=20)

        # Put legend
        plt.legend(fontsize=8)
            
        # Export the current page
#         plt.show()
        f.savefig(filename+elem_names[i]+".pdf")
        plt.close(f)

In [15]:
plot_mg_stack(old_spec,new_spec,filename='norm_spec_plot/stack_plots/stack_',start_wls=start_wls,end_wls=end_wls,elem_names=elem_names)